<a href="https://colab.research.google.com/github/farzanehnia/PYOMO/blob/main/MILP_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install pyomo

Here is an example of MILP mixed integer linear programming
Solvers include: ['gurobi', 'cplex', 'cbc', 'glpk', 'gams', 'gurobi_persistent', 'cplex_persistent'] for linear programming
GLOP is a google software used in or-tools not PYOMO.

mixed-integer nonlinear programming problems (MINLP), ipopt(NLP)

In [ ]:
!pip install -q pyomo

     |████████████████████████████████| 9.6 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 


In [ ]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import time
import numpy as np

In [ ]:
#model
model = pyo.ConcreteModel()

model.x = pyo.Var(range(1, 6), within=NonNegativeIntegers)

model.y = pyo.Var(bounds=(0, None))

x = model.x
y = model.y

model.x.pprint()


x : Size=5, Index=x_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  None :  None : False :  True : NonNegativeIntegers
      2 :     0 :  None :  None : False :  True : NonNegativeIntegers
      3 :     0 :  None :  None : False :  True : NonNegativeIntegers
      4 :     0 :  None :  None : False :  True : NonNegativeIntegers
      5 :     0 :  None :  None : False :  True : NonNegativeIntegers


In [ ]:
#Constraints:
Nx = len(x)
sumx = sum(  [ x[i] for i in range(1, Nx+1) ]  )
model.C1 = pyo.Constraint( expr =  sumx +  y  <= 20  )

model.C2 = pyo.ConstraintList()
for i in range(1, Nx+1):
  model.C2.add(expr = x[i]+y >= 15)

sum2 = sum([x[i]*i for i in range(1, len(x)+1)])
model.C3 = pyo.Constraint( expr = sum2  >=10  )

model.C4 =  pyo.Constraint( expr = x[5]+2*y >= 30 )


In [ ]:
model.obj = pyo.Objective(expr = sumx + y, sense=minimize)

In [ ]:
!apt install glpk-utils
!pip install glpk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
glpk-utils is already the newest version (4.65-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached glpk-0.4.6.tar.gz (153 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  ERROR: Failed building wheel for glpk
Failed to build glpk
ERROR: Could not build wheels for glpk which use PEP 517 and cannot be installed directly


In [ ]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155842 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [ ]:
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [ ]:
!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
!unzip -o -q couenne-linux64

In [ ]:

begin = time.time()

opt = SolverFactory('cbc', executable='/usr/bin/cbc')    # CBC, Gurobi, Cplex, Glop --> from google
results = opt.solve(model)

duration = time.time() - begin
print('Time passed = {}'.format(duration))


Time passed = 0.054369211196899414


In [ ]:

begin = time.time()

opt = SolverFactory('glpk')    
results = opt.solve(model)

duration = time.time() - begin
print('Time passed = {}'.format(duration))

Time passed = 0.02507781982421875


In [ ]:
begin = time.time()

opt = SolverFactory('ipopt', executable='/content/ipopt')
results = opt.solve(model)

duration = time.time() - begin
print('Time passed = {}'.format(duration))

Time passed = 0.031111478805541992


In [ ]:
begin = time.time()

opt = SolverFactory('couenne', executable='/content/couenne')
results = opt.solve(model)

duration = time.time() - begin
print('Time passed = {}'.format(duration))

Time passed = 0.05847978591918945


In [ ]:
for i in range(1,6):
  print('x[{}] = {}'.format(i, pyo.value(x[i])))
print('y= {}'.format(pyo.value(y)))
print('Objective function value ={}'.format(pyo.value(model.obj)))

x[1] = 0.0
x[2] = 0.0
x[3] = 0.0
x[4] = 0.0
x[5] = 2.0
y= 15.0
Objective function value =17.0
